In [1]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from vehi_rout.utils.helper_utils import get_penalty_list, get_values_not_in_second_list
from vehi_rout.utils.visualization import visualize_routes_per_vehicle
from vehi_rout.utils.data_utils import get_demand_df, update_demand_dic, load_matrix_df, get_demand_matrix_df, load_df
from vehi_rout.utils.route_utils import sort_nodes_by_distance
from vehi_rout.constant import *
from vehi_rout.utils.visualization import visualize_routes_per_vehicle, save_route_details_to_csv, print_route_summary

load c:\Users\HP\Desktop\E-Vision-Projects\Vehicle_Routing\venv\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\HP\Desktop\E-Vision-Projects\Vehicle_Routing\venv\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\HP\Desktop\E-Vision-Projects\Vehicle_Routing\venv\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\HP\Desktop\E-Vision-Projects\Vehicle_Routing\venv\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\HP\Desktop\E-Vision-Projects\Vehicle_Routing\venv\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\HP\Desktop\E-Vision-Projects\Vehicle_Routing\venv\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\HP\Desktop\E-Vision-Projects\Vehicle_Routing\venv\Lib\site-packages\ortools\.libs\ortools.dll...
Loaded 1610 cached routes from ../data/csv/route_cache.csv


In [380]:
TOTAL_DAYS = 6
MAX_VISITS_PER_VEHICLE = [12]*2 + [12]*2 #+ [15]*2 + [25]*2

DISTANCE_BASE_PENALTY = 1000
MAX_DISTANCE_PER_VEHICLE = [150]*2 + [150]*2 #+ [600]*2 + [600]*2

DEPOT = 0

In [324]:
def print_solution(manager, routing, solution, data, day):
    total_distance = 0  # Changed from total_time to total_distance
    visited_nodes = set()
    route_dict = {}  # Dictionary to store route details for each vehicle
    
    print(f"\nDay {day + 1} Routes (Penalty per unvisited demand unit: {data['penalties'][1]} units):")
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0  # Changed from route_time to route_distance
        num_visits = 0
        route_nodes = []  # List to store the sequence of nodes for this vehicle
        previous_node = None
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            original_node = data["node_mapping"][node]
            visited_nodes.add(original_node)
            route_nodes.append(original_node)
            plan_output += f" {original_node} ->"
            if previous_node is not None:
                arc_distance = int(data["distance_matrix"][previous_node][node])  # Changed from time_matrix to distance_matrix
                route_distance += arc_distance
            previous_node = node
            index = solution.Value(routing.NextVar(index))
            if original_node != data["depot"]:
                num_visits += 1
        
        node = manager.IndexToNode(index)
        original_node = data["node_mapping"][node]
        visited_nodes.add(original_node)
        route_nodes.append(original_node)
        plan_output += f" {original_node}\n"
        if previous_node is not None:
            arc_distance = int(data["distance_matrix"][previous_node][node])  # Changed from time_matrix to distance_matrix
            route_distance += arc_distance
        
        plan_output += f"Distance of the route: {route_distance} km\n"  # Changed from Time to Distance, assumed km
        max_distance = data["max_distance_per_vehicle"][vehicle_id]  # Changed from max_time_per_vehicle to max_distance_per_vehicle
        plan_output += f"Within limit: {'Yes' if route_distance <= max_distance else 'No'} (Max: {max_distance} km)\n"  # Changed units to km
        plan_output += f"Stops visited: {num_visits-1}/{data['max_visits_per_vehicle'][vehicle_id]}\n"
        
        # Store route details in the dictionary
        route_dict[vehicle_id] = {
            "route_nodes": route_nodes,
            "route_distance": route_distance,  # Changed from route_time to route_distance
            "max_distance_limit": max_distance,  # Changed from max_time_limit to max_distance_limit
            "within_limit": route_distance <= max_distance,
            "num_visits": num_visits,
            "max_visits_limit": data["max_visits_per_vehicle"][vehicle_id]
        }
        
        print(plan_output)
        total_distance = max(total_distance, route_distance)  # Changed from total_time to total_distance
    
    print(f"Maximum route distance for Day {day + 1}: {total_distance} km")  # Changed from time to distance
    
    return visited_nodes, route_dict  # Return both visited_nodes and route_dict

In [4]:
# In data_model.vrp_data_model.py
def create_data_model(full_matrix, nodes_to_visit, demand_dict, penalty_lis, use_distance=False):
    data = {}
    node_indices = [0] + [i for i, code in enumerate(full_matrix.index) if code in demand_dict['key']]
    nodes_to_use = [node_indices[0]] + [i for i in node_indices[1:] if i in nodes_to_visit]
    data["num_vehicles"] = len(MAX_DISTANCE_PER_VEHICLE)
    data["depot"] = 0
    if use_distance:
        # Construct distance matrix from full_matrix
        data["distance_matrix"] = [[full_matrix.iloc[i][j] for j in nodes_to_use] for i in nodes_to_use]
        data["max_distance_per_vehicle"] = MAX_DISTANCE_PER_VEHICLE  
    else:
        # Original time matrix (for backward compatibility)
        data["time_matrix"] = [[full_matrix.iloc[i][j] for j in nodes_to_use] for i in nodes_to_use]
        data["max_time_per_vehicle"] = MAX_TIME_PER_VEHICLE 
        
    data["demands"] = [0] + [demand_dict.get(full_matrix.index[i], 1) for i in nodes_to_use[1:]]  # Demand at each node
    data["node_mapping"] = [full_matrix.index[i] for i in nodes_to_use] #{i: node for i, node in enumerate([0] + nodes_to_visit)}  # Mapping of indices to nodes
    data["max_visits_per_vehicle"] = MAX_VISITS_PER_VEHICLE 
    data["penalties"] = [0] + penalty_lis
    return data

In [5]:
def solve_vrp_for_day(full_matrix, nodes_to_visit, day, demand_dict, penalty_lis):
    # Assuming create_data_model now returns a distance_matrix instead of time_matrix
    data = create_data_model(full_matrix, nodes_to_visit, demand_dict,penalty_lis, use_distance=True)  # Adjust create_data_model
    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        distance = int(data["distance_matrix"][from_node][to_node])  # Use distance instead of time
        return distance

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Distance dimension (replacing Time dimension)
    routing.AddDimension(
        transit_callback_index,
        0,
        max(data["max_distance_per_vehicle"]),  # New max distance limit per vehicle
        True,
        "Distance"
    )
    distance_dimension = routing.GetDimensionOrDie("Distance")
    for vehicle_id in range(data["num_vehicles"]):
        end_index = routing.End(vehicle_id)
        distance_dimension.CumulVar(end_index).SetMax(data["max_distance_per_vehicle"][vehicle_id])  # Set distance limit

    # Demand dimension remains unchanged
    def demand_callback(from_index):
        return data["demands"][manager.IndexToNode(from_index)]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,
        data["max_visits_per_vehicle"],
        True,
        "Visits"
    )

    # Optional nodes with demand-based penalty (unchanged)
    for node in range(1, len(data["distance_matrix"])):
        routing.AddDisjunction([manager.NodeToIndex(node)], data["penalties"][node])

    # Search parameters (unchanged)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 30

    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        return print_solution(manager, routing, solution, data, day)
    else:
        print(f"No solution found for Day {day + 1}!")
        return set()

In [353]:
demand_df = get_demand_df(
     today_path='../data/orders/07-03-2025-PO.csv',
    #  wait_path='../data/orders/next_day_demand.csv'
    )

demand_df.shape

(23, 8)

In [354]:
if demand_df['CODE'].dtype in ['float', 'int', 'int64']:
    demand_df['CODE'] = demand_df['CODE'].astype(int)
    demand_df['CODE'] = demand_df['CODE'].astype(str)
    print('Converting to Object')

Converting to Object


In [355]:
master_mat_df = load_matrix_df(path='../data/master/osrm_distance_matrix.csv')

In [356]:
demand_mat_df = get_demand_matrix_df(master_mat_df,demand_df, 0)

In [357]:
demand_mat_df.shape

(24, 24)

In [358]:
demand_dict = update_demand_dic(demand_df)

In [359]:
len(demand_dict['key'])

23

In [360]:
len(demand_dict['po_date'])

23

In [381]:
today = '2025-03-06'
penalty_lis = get_penalty_list(demand_dict, DISTANCE_BASE_PENALTY, TOTAL_DAYS, today)

Penalty for 1160: 500
Penalty for 1585: 500
Penalty for 1658: 500
Penalty for 1673: 500
Penalty for 1684: 500
Penalty for 1686: 500
Penalty for 1687: 500
Penalty for 1688: 500
Penalty for 1689: 500
Penalty for 1690: 500
Penalty for 1691: 500
Penalty for 1692: 500
Penalty for 1693: 500
Penalty for 1696: 500
Penalty for 1860: 500
Penalty for 1868: 500
Penalty for 1873: 500
Penalty for 1880: 500
Penalty for 1881: 500
Penalty for 1883: 500
Penalty for 1884: 500
Penalty for 1886: 500
Penalty for 1889: 500


In [362]:
# new_lis = []
# for i in penalty_lis:
#     new_lis.append(int(i)*100)

In [363]:
# penalty_lis = new_lis

In [382]:
demand_sort_nodes = sort_nodes_by_distance(demand_mat_df.values)

In [383]:
nodes_to_visit = sorted(demand_sort_nodes[:])

In [384]:
visited_node, route_dict = solve_vrp_for_day(demand_mat_df, nodes_to_visit, 1, demand_dict, penalty_lis)


Day 2 Routes (Penalty per unvisited demand unit: 500 units):
Route for vehicle 0:
 0 -> 0
Distance of the route: 0 km
Within limit: Yes (Max: 150 km)
Stops visited: 0/12

Route for vehicle 1:
 0 -> 0
Distance of the route: 0 km
Within limit: Yes (Max: 150 km)
Stops visited: 0/12

Route for vehicle 2:
 0 -> 1690 -> 1873 -> 0
Distance of the route: 81 km
Within limit: Yes (Max: 150 km)
Stops visited: 2/12

Route for vehicle 3:
 0 -> 1673 -> 1886 -> 1692 -> 1889 -> 1658 -> 1160 -> 1691 -> 1693 -> 1881 -> 0
Distance of the route: 93 km
Within limit: Yes (Max: 150 km)
Stops visited: 9/12

Maximum route distance for Day 2: 93 km


In [385]:
not_visited_codes = get_values_not_in_second_list(demand_df['CODE'], list(visited_node))

not_visited_codes_str = list(map(str, not_visited_codes))
# Filter the dataframe
sub_df = demand_df[demand_df['CODE'].isin(not_visited_codes_str)]
print(sub_df.shape)
# sub_df.to_csv('../data/orders/next_day_demand.csv', index=False)

(12, 8)


In [386]:
import folium
import pandas as pd

def visualize_all_locations(df):
    """
    Visualize all locations from a CSV file on a folium map.
    :param csv_file_path: Path to the CSV file containing location data
    :return: folium.Map object
    """
    # Load the CSV data into a DataFrame
    # try:
    #     df = df
    # except Exception as e:
    #     print(f"Error loading CSV file {csv_file_path}: {e}")
    #     return None
    try:
        # Create a map centered at the mean of all latitudes and longitudes
        map_center = [df['LATITUDE'].mean(), df['LONGITUDE'].mean()]
        m = folium.Map(location=map_center, zoom_start=10)
    except Exception as e:
        print(f"Error getting gps data: {e}")
        return None

    # Add a marker for each location
    for _, row in df.iterrows():
        folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            popup=f"CODE: {row['CODE']}<br>LOCATION: {row['LOCATION']}<br>ADDRESS: {row['ADDRESS']}<br>BRAND: {row['BRAND']}",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

    # Add title
    title_html = '<h3 align="center" style="font-size:16px">All Locations Map</h3>'
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [387]:
visualize_all_locations(sub_df)

In [388]:
master_gps_df = load_df('../data/master/master_gps.csv')

In [390]:
total_distance = 0
for veh in route_dict.keys():
    print(f"Vehicle id: {veh}")
    path = ''
    for code in route_dict[veh]['route_nodes']:
        if code == '0':
            loc_name = "SMAK"
        else:
            loc = master_gps_df.loc[master_gps_df['CODE']==code]
            if len(loc) is None:
                loc_name = ["SMAK"]
            loc_name = loc['LOCATION'].values[0]
        
        path += f'{loc_name} ({code}) --> '
    path += f"Route Distance: {route_dict[veh]['route_distance']}km (Max: {route_dict[veh]['max_distance_limit']}km) Visits: {route_dict[veh]['num_visits']-1} (Max: {route_dict[veh]['max_visits_limit']})"
    total_distance += route_dict[veh]['route_distance']
    print(path)
    print('\n')
print(f"Total distance for all vehicles: {total_distance}km")

Vehicle id: 0
SMAK (0) --> SMAK (0) --> Route Distance: 0km (Max: 150km) Visits: 0 (Max: 12)


Vehicle id: 1
SMAK (0) --> SMAK (0) --> Route Distance: 0km (Max: 150km) Visits: 0 (Max: 12)


Vehicle id: 2
SMAK (0) --> FC VEYANGODA (1690) --> EX KANDALAMA (MIRIGAMA) (1873) --> SMAK (0) --> Route Distance: 81km (Max: 150km) Visits: 2 (Max: 12)


Vehicle id: 3
SMAK (0) --> FC WALPOLA (1673) --> EX CAPITAL HEIGHTS (1886) --> FC TORRINGTON SQUARE (1692) --> EX WORLD TRADE CENTRE (1889) --> FH HAVELOCK CITY (1658) --> FC ATTIDIYA (1160) --> FC MOUNT LAVINIA 02 (1691) --> FC KATUBEDDA (SQUARE) (1693) --> EX ATIGALA (1881) --> SMAK (0) --> Route Distance: 93km (Max: 150km) Visits: 9 (Max: 12)


Total distance for all vehicles: 174km


In [372]:
import pandas as pd

In [373]:
SMAK_KADAWATHA = (7.0038321,79.9394804)

smak_data = {
    "CODE":'0',
    "LOCATION":"SMAK",
    "ADDRESS":"Smak, Kadawatha, Western Province, Sri Lanka",
    "LATITUDE":SMAK_KADAWATHA[0],
    "LONGITUDE":SMAK_KADAWATHA[1]
}


master_df = pd.concat(
    [
        pd.DataFrame(smak_data, index=[0]),
        master_gps_df
    ],
    ignore_index=True
)

In [391]:
map_obj_2 = visualize_routes_per_vehicle(master_df, route_dict, 1)

Processing route for vehicle 0: ['0', '0']
Using cached path for 0 to 0
Skipping path from 0 to 0 — insufficient points (2 points)
No path coordinates available for vehicle 0. Route not plotted.
Processing route for vehicle 1: ['0', '0']
Using cached path for 0 to 0
Skipping path from 0 to 0 — insufficient points (2 points)
No path coordinates available for vehicle 1. Route not plotted.
Processing route for vehicle 2: ['0', '1690', '1873', '0']
Using cached path for 0 to 1690
Added path from 0 to 1690 with 953 points
Using cached path for 1690 to 1873
Added path from 1690 to 1873 with 341 points
Fetching path from 1873 ((7.2427345, 80.1269539)) to 0 ((7.0038321, 79.9394804))
Path cached for 1873 to 0: [[7.242728, 80.126971], [7.242624, 80.126923], [7.242581, 80.126923], [7.242552, 80.126942], [7.242467, 80.12711]]... (total 1258 points)
Added path from 1873 to 0 with 1258 points
Plotted path for vehicle 2 with 2552 coordinates
Processing route for vehicle 3: ['0', '1673', '1886', '1692

In [392]:
map_obj_2[0]

In [393]:
map_obj_2[1]

In [394]:
map_obj_2[2]

In [395]:
map_obj_2[3]

In [379]:
map_obj_2[4]

KeyError: 4

In [ ]:
map_obj_2[5]

In [ ]:
map_obj_2[6]

KeyError: 6

In [ ]:
map_obj_2[7]

KeyError: 7